In [1]:
import os
import pickle as pkl
import numpy as np
import pandas as pd
import joblib
import __init__

Navigated to package root: /home/cyprien/CrystaLLMv2_PKV
Added package root to Python path


In [ ]:
# Make prompts from the generated CIFs
!python _utils/_generating/make_prompts.py \
    --manual \
    --compositions "Na1Cl1, Cl2Fe4" \
    --condition_lists "1.2,2.5" "0.1,0.5" \
    --output_parquet 'test.parquet'


Generated manual prompts for 2 compositions and 2 condition lists

First 3 rows:
                Prompt condition_vector
<bos>\ndata_[Na1Cl1]\n         1.2, 0.1
<bos>\ndata_[Na1Cl1]\n         1.2, 0.5
<bos>\ndata_[Na1Cl1]\n         2.5, 0.1

Saved 8 prompts to test.parquet


In [12]:
!python _utils/_generating/make_prompts.py \
    --automatic \
    --HF_dataset 'c-bone/ricci_pf_2' \
    --split 'test' \
    --cif_column 'CIF' \
    --level 'level_2' \
    --condition_columns 'norm_pf_at_700K' 'gap' \
    --output_parquet 'test.parquet' \
    --remove_ref_columns

Loaded 853 records from c-bone/ricci_pf_2 (test split)

Generated automatic prompts at level_2
Using condition columns: ['norm_pf_at_700K', 'gap']

Removed reference columns, keeping only: ['Prompt', 'condition_vector']

First 3 rows:
                      Prompt          condition_vector
      <bos>\ndata_[Sm1Th1]\n              15.0209, 0.0
 <bos>\ndata_[Ba2Er1Cu3O7]\n              14.7939, 0.0
<bos>\ndata_[Li6Ni2B10O20]\n 14.5722, 4.09030637629976

Saved 853 prompts to test.parquet


In [28]:
# Generating CIFs from test 
!python _utils/_generating/generate_CIFs.py \
    --config '_config_files/cg_eval/test_eval.jsonc'

Environment info
Available GPUs: 2
GPU 0: NVIDIA L4
GPU 1: NVIDIA L4

Generation settings
Total sequences per prompt-condition pair: 1000
Will save generated CIFs to test.parquet
Using model checkpoint: model_ckpts/mpdb_2prop_scratch/PKV/checkpoint-10500

Model checkpoint info
Most Recent Train Loss: 0.6455
Most Recent Validation Loss: 0.3434
Model's max_length: 1026
Tokenizer validation passed: token vocabulary is consistent.
Generation kwargs: {'max_length': 1018, 'pad_token_id': 371, 'eos_token_id': 373, 'renormalize_logits': True, 'remove_invalid_values': True, 'num_return_sequences': 25, 'do_sample': True, 'top_k': 30, 'top_p': 0.95, 'temperature': 0.7}

Generation
Number of condition-prompt pairs: 8
Tokenizer validation passed: token vocabulary is consistent.
Loading model with PKV conditionality...
Generating CIFs...:   0%|                              | 0/8000 [00:00<?, ?it/s]Tokenizer validation passed: token vocabulary is consistent.
Loading model with PKV conditionality...
T

In [ ]:
!python _utils/_generating/evaluate_CIFs.py \
    --input_parquet 'test.parquet' \
    --metrics_out None \
    --num_workers 8  

Loaded 8000 CIFs from test.parquet
Tokenizer validation passed: token vocabulary is consistent.
100%|███████████████████████████████████████| 8000/8000 [07:18<00:00, 18.23it/s]

space group consistent: 7767/8000 (0.971)
atom site multiplicity consistent: 7758/8000 (0.970)
avg. bond length reasonableness score: 0.8905 ± 0.1604
bond lengths reasonable: 3808/8000 (0.476)

num valid: 3775/8000 (0.47)
longest valid generated length: 936
avg. valid generated length: 419.325 ± 89.569


In [32]:
!python _utils/_generating/postprocess.py \
    --input_parquet 'test.parquet' \
    --output_parquet 'test.parquet' \
    --num_workers 1

Processing 8000 records using 1 worker(s)
100%|███████████████████████████████████████| 8000/8000 [05:42<00:00, 23.38it/s]
Single-worker processing completed
